In [15]:
import psycopg2


In [36]:
connection = psycopg2.connect(
  dbname="trigram",
  user="postgres",
  password="postgres",
  host="localhost",
  port="5432"
)

db = connection.cursor()

# Setup

In [53]:
db.execute("CREATE EXTENSION IF NOT EXISTS pg_trgm;")

In [ ]:
db.execute("""
  CREATE TABLE IF NOT EXISTS documents (
    id SERIAL PRIMARY KEY,
    title VARCHAR(255) NOT NULL,
    language CHAR(2) NOT NULL,
    content TEXT NOT NULL,
    summary TEXT NOT NULL
  );
""")


# Random documents

In [55]:
db.execute("""
  INSERT INTO documents (title, language, content, summary) VALUES
    ('Random Document 1', 'en', 'This is the content of the first random document.', 'A brief summary of the first document.'),
    ('Random Document 2', 'es', 'Este es el contenido del segundo documento aleatorio.', 'Un breve resumen del segundo documento.'),
    ('Random Document 3', 'fr', 'Voici le contenu du troisième document aléatoire.', 'Un bref résumé du troisième document.'),
    ('Random Document 4', 'de', 'Dies ist der Inhalt des vierten zufälligen Dokuments.', 'Eine kurze Zusammenfassung des vierten Dokuments.'),
    ('Random Document 5', 'it', 'Questo è il contenuto del quinto documento casuale.', 'Un breve riassunto del quinto documento.');
""")

In [69]:
db.execute("SELECT * FROM documents;")
db.fetchall()


[(1,
  'Random Document 1',
  'en',
  'This is the content of the first random document.',
  'A brief summary of the first document.'),
 (2,
  'Random Document 2',
  'es',
  'Este es el contenido del segundo documento aleatorio.',
  'Un breve resumen del segundo documento.'),
 (3,
  'Random Document 3',
  'fr',
  'Voici le contenu du troisième document aléatoire.',
  'Un bref résumé du troisième document.'),
 (4,
  'Random Document 4',
  'de',
  'Dies ist der Inhalt des vierten zufälligen Dokuments.',
  'Eine kurze Zusammenfassung des vierten Dokuments.'),
 (5,
  'Random Document 5',
  'it',
  'Questo è il contenuto del quinto documento casuale.',
  'Un breve riassunto del quinto documento.')]

# Index setup

In [ ]:
db.execute("""
  CREATE INDEX IF NOT EXISTS trgm_summary_idx ON documents USING GIN (summary gin_trgm_ops);
  CREATE INDEX IF NOT EXISTS trgm_content_idx ON documents USING GIN (content gin_trgm_ops);
""")


In [75]:
db.execute("SET pg_trgm.similarity_threshold = 0.1;")

# Search

In [76]:
query = "document"

db.execute("""
  SELECT *, similarity(summary, %s) AS similarity_score
  FROM documents
  WHERE summary %% %s
  ORDER BY summary <-> %s
""", (query, query, query))

results = db.fetchall()
results


[(3,
  'Random Document 3',
  'fr',
  'Voici le contenu du troisième document aléatoire.',
  'Un bref résumé du troisième document.',
  0.25),
 (1,
  'Random Document 1',
  'en',
  'This is the content of the first random document.',
  'A brief summary of the first document.',
  0.23684211),
 (5,
  'Random Document 5',
  'it',
  'Questo è il contenuto del quinto documento casuale.',
  'Un breve riassunto del quinto documento.',
  0.22222222),
 (2,
  'Random Document 2',
  'es',
  'Este es el contenido del segundo documento aleatorio.',
  'Un breve resumen del segundo documento.',
  0.21621622)]